In [1]:
from gensim import models
from sklearn.linear_model import LogisticRegressionCV
from doc2vec.doc2vec import doc2vec

C:\Users\nebboy4\Anaconda3\envs\github_env\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Load Doc2Vec Model & Get Vectors

In [2]:
d2v_model = doc2vec(model=models.Doc2Vec.load("../data/scotus/scotus_model.doc2vec"),label_docs=False)

In [3]:
embeddings = d2v_model.model.docvecs

In [4]:
names = []
vectors = []
for tag,_ in embeddings.doctags.items():
    names.append(tag)
    vectors.append(embeddings[tag].tolist())
print('complete')

complete


In [5]:
import random

In [6]:
x = [[random.uniform(0,100) for _ in range(300)] for _i_ in range(30000)]
y = [int(random.uniform(0,1) > 0.5) for _ in range(30000)]
a = LogisticRegressionCV([0.25,0.5,1.0,2.0,10.0,100.0],
                         penalty='l2',
                         scoring='accuracy',
                         solver='sag',
                         n_jobs=-1,
                         verbose=1,
                         refit=True,
                         multi_class='multinomial'
                        )

In [7]:
a.fit(x,y)

max_iter reached after 22 seconds
max_iter reached after 22 seconds
max_iter reached after 23 seconds
max_iter reached after 30 seconds
max_iter reached after 31 seconds
max_iter reached after 31 seconds
max_iter reached after 25 seconds
max_iter reached after 24 seconds
max_iter reached after 24 seconds
max_iter reached after 21 seconds
max_iter reached after 22 seconds
max_iter reached after 22 seconds
max_iter reached after 24 seconds
max_iter reached after 23 seconds
max_iter reached after 23 seconds
max_iter reached after 21 seconds
max_iter reached after 21 seconds
max_iter reached after 21 seconds


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.4min finished


LogisticRegressionCV(Cs=[0.25, 0.5, 1.0, 2.0, 10.0, 100.0], class_weight=None,
           cv=None, dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=100, multi_class='multinomial', n_jobs=-1,
           penalty='l2', random_state=None, refit=True, scoring='accuracy',
           solver='sag', tol=0.0001, verbose=1)